In [136]:
from sklearn.datasets import fetch_covtype
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (fetch_covtype())

x = pd.DataFrame(datasets["data"])
y = (datasets["target"])




In [137]:

# y를 ndarry 로 변환해줌
# 또는 y = y.values 사용가능
# y = y.to_numpy()


(581012,)
(581012, 1)


In [148]:
from sklearn.preprocessing import OneHotEncoder
# 원핫 인코더를 적용하기전 쉐이프 변환
print(y.shape)
y = y.reshape(581012, 1)
print(y.shape)

# 원핫 인코더 변환
ohe = OneHotEncoder()
ohe.fit(y)
y = ohe.transform(y)

# 넘파이 변환
y= y.toarray()

(581012, 7)


ValueError: cannot reshape array of size 4067084 into shape (581012,1)

In [139]:
print(x.shape,y.shape) 

print(np.unique(y)) # [1 2 3 4 5 6 7]
# 불균형확인
# 참고로 이 y 값은 0 클래스가 없기 대문에 0 을 지워줘야합니다 ... ㅂㄷㅂㄷ

print(np.unique(y,return_counts=True)) # [211840, 283301,  35754,   2747,   9493,  17367,  20510




(581012, 54) (581012, 7)
[0. 1.]
(array([0., 1.]), array([3486072,  581012], dtype=int64))


In [140]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(
                                                    x,
                                                    y,
                                                    train_size=0.8,
                                                    shuffle = True,
                                                    #  stratify 는 데이터 불균형을 해결해줌
                                                    stratify=y,
                                                    random_state=21
                                                   )
# print(x.columns)
# y 를 원 핫 인코딩 변환

print(y_train[:10])
print("y_train shape : ",y_train.shape)
print("y_test shape : ",y_test.shape)

[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]]
y_train shape :  (464809, 7)
y_test shape :  (116203, 7)


In [141]:
model = Sequential([
    Dense(128,activation="relu",input_shape=(54,)),
    Dense(128,activation="relu") ,
    Dense(128,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(32,activation="relu") ,
    Dense(32,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(7,activation="softmax")
])

In [142]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
patience=10, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=10, 
                 validation_split=0.2,batch_size=256,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/10
1453/1453 [==============================] - 6s 4ms/step - loss: 1.0927 - accuracy: 0.6159 - val_loss: 0.7012 - val_accuracy: 0.6994
Epoch 2/10
1453/1453 [==============================] - 5s 4ms/step - loss: 0.6715 - accuracy: 0.7110 - val_loss: 0.6539 - val_accuracy: 0.7273
Epoch 3/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.6161 - accuracy: 0.7341 - val_loss: 0.6207 - val_accuracy: 0.7351
Epoch 4/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.5872 - accuracy: 0.7459 - val_loss: 0.5423 - val_accuracy: 0.7686
Epoch 5/10
1453/1453 [==============================] - 5s 4ms/step - loss: 0.5573 - accuracy: 0.7588 - val_loss: 0.5403 - val_accuracy: 0.7685
Epoch 6/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.5298 - accuracy: 0.7704 - val_loss: 0.5053 - val_accuracy: 0.7865
Epoch 7/10
1453/1453 [==============================] - 6s 4ms/step - loss: 0.5036 - accuracy: 0.7832 - val_loss: 0.4955 - val_accuracy:

In [143]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


3632/3632 [==============================] - 7s 2ms/step - loss: 0.4345 - accuracy: 0.8132
loss: 0.43449732661247253 
acc : 0.8131889700889587


In [144]:
from sklearn.metrics import accuracy_score

y_predict = model.predict(x_test)
print(y_predict)
y_predict = np.argmax(y_predict,axis=1)


[[4.2905736e-01 5.6265664e-01 1.8176300e-07 ... 8.0026770e-03
  2.6336351e-05 2.5636677e-04]
 [3.3784769e-02 9.6190673e-01 1.2534857e-05 ... 4.2662029e-03
  4.3991081e-06 2.5363744e-05]
 [5.5601635e-07 2.5900303e-05 9.9244457e-01 ... 9.2245791e-08
  7.5205984e-03 1.1170552e-12]
 ...
 [1.6544556e-04 7.5409859e-02 1.8950361e-04 ... 9.2423487e-01
  6.1649828e-08 2.5532216e-07]
 [5.5240463e-02 9.4461328e-01 5.0146311e-05 ... 8.1308090e-06
  3.1046766e-05 5.6987483e-05]
 [3.5031226e-01 6.4965200e-01 3.9674887e-08 ... 1.9418279e-05
  3.4344043e-06 1.2850412e-05]]


In [145]:
print(y_predict)

y_predict = tf.one_hot(y_predict,depth =7)


[1 1 2 ... 4 1 1]


In [146]:
# 데이터 출력

print("예측데이터 : ",y_predict[:20])
print("실제데이터 : ",y_test[:20])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)


예측데이터 :  tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]], shape=(20, 7), dtype=float32)
실제데이터 :  [[0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0

In [147]:
model.save("./_data\\"+"fetch(val_acc=0.89).h5")